<a href="https://colab.research.google.com/github/ougrid/my-knowledge-resource/blob/master/finetuning_xlm_withprompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Plan & Recommendation

In [ ]:
#ปรับpromt
#tagชื่อ
#translate
#มองดูว่าคำถามตรงไหนmatchกัน
#ใช้pattern แยกคำถามด้วยrule baseก่อน

#Import Librery

In [ ]:
#import basic librery
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split #how to split data

#basic torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import re
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from transformers import AutoModelForSequenceClassification
import ast
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from torch.jit import script, trace       # hybrid frontend decorator and tracing jit
from sklearn.metrics import f1_score
from datetime import datetime
import random
import time
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup, TFXLMRobertaModel,  XLMRobertaForSequenceClassification

#classic tool & classic ML
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC


2024-05-09 15:35:13.108344: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 15:35:13.108400: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 15:35:13.109895: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM


In [ ]:
!pip install transformers

In [ ]:
!pip install transformers[tf-cpu]

In [ ]:
!pip install pythainlp -q

In [ ]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,

)

In [ ]:
#Import data set
train_df = pd.read_csv('/kaggle/input/super-train-title/super_train_title.csv', index_col = 0)
#test_df = pd.read_csv('test_add_tag.csv', index_col = 0)
# pat_df = pd.read_csv('/kaggle/input/legal-act-classification/patterns.csv')

In [ ]:
#create prompt

prompts = []
for index, row in train_df.iterrows():
    prompt = f'''
    คุณเป็นผู้ช่วยทางด้านนิติกรรม คุณจะต้องตอบคำถามทางด้านนิติกรรมโดยใช้ข้อมูลต้นฉบับเท่านั้น\n
    ข้อมูลต้นฉบับ : {row['context']} \n
    คำถาม : คุณ {row['question']} จะทำการ {row['legal_act']} ทำได้หรือไม่? ตอบแค่ได้ หรือ ไม่ได้ ถ้าได้ให้ตอบ 1 และถ้าไม่ได้ให้ตอบ 0'''
    prompts.append(prompt)

# Create a new column named 'prompt' with the generated prompts
train_df['prompt'] = prompts


In [ ]:
train_df.to_csv("/kaggle/working/fortranslate.csv")

In [ ]:
len(prompts)

4429

In [ ]:
train_df.keys()

Index(['id', 'rgno', 'context', 'pattern', 'question', 'legal_act',
       'condition', 'answer', 'prompt'],
      dtype='object')

In [ ]:
#train_df = train_df.drop(columns=['prompt'])


In [ ]:
#data split

X = train_df['prompt']  # Your features
y = train_df['answer']  # Your target labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#convert into PyTorch Datasets
#train_data = TensorDataset(X_train,y_train)

#translate into dataloader objects
#batchsize    = 8
#train_loader = DataLoader(train_data,batch_size=batchsize,shuffle=True,drop_last=True)

#print(f"Training set size: {len(train_data)}")

#print(f"Validation set shape: {X_test.shape}")

In [ ]:
#load model
models = [
    AutoModelForSequenceClassification.from_pretrained(
        "airesearch/wangchanberta-base-att-spm-uncased",
        #num_labels=2,  # Assuming binary classification
        #hidden_dropout_prob=0.2
    ),
    #AutoModelForSequenceClassification.from_pretrained(
    AutoModelForSequenceClassification.from_pretrained(
        'FacebookAI/xlm-roberta-base'#,
        #hidden_size=512,  # Example: customize hidden size
        #num_attention_heads=8
    ),]

#choose tokenizer
tokenizers = [
    AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion"),
    AutoTokenizer.from_pretrained('FacebookAI/xlm-roberta-base',  problem_type="multi_label_classification"),
]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model =  AutoModelForSequenceClassification.from_pretrained('FacebookAI/xlm-roberta-base')
tokenizer = AutoTokenizer.from_pretrained('FacebookAI/xlm-roberta-base',  problem_type="multi_label_classification")

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# df_train.review = df_train.str.lower()
sentences = train_df.prompt.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = train_df.answer.values

In [ ]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])

Token indices sequence length is longer than the specified maximum sequence length for this model (761 > 512). Running this sequence through the model will result in indexing errors


Tokenize the first sentence:
['▁[', 'C', 'LS', ']', '▁คุณ', 'เป็น', 'ผู้ช่วย', 'ทางด้าน', 'นิติ', 'กรรม', '▁คุณ', 'จะต้อง', 'ตอบ', 'คําถาม', 'ทางด้าน', 'นิติ', 'กรรม', 'โดยใช้', 'ข้อมูล', 'ต้น', 'ฉบับ', 'เท่านั้น', '▁ข้อมูล', 'ต้น', 'ฉบับ', '▁:', '▁', 'กรรม', 'การ', 'คน', 'ใด', 'คนหนึ่ง', 'ลง', 'ลาย', 'มือ', 'ชื่อ', 'ร่วมกับ', 'กรรม', 'การ', 'อื่น', 'อีกหนึ่ง', 'คน', 'รวม', 'เป็น', 'สอง', 'คน', 'และ', 'ประ', 'ทับ', 'ตรา', 'สําคัญ', 'ของ', 'บริษัท', '▁', 'คําถาม', '▁:', '▁คุณ', '▁[', "'", 'P', '000', '6', "'", ']', '▁จะ', 'ทําการ', '▁', 'การทํา', 'นิติ', 'กรรม', '▁สํานักงาน', 'ตรวจ', 'คน', 'เข้า', 'เมือง', '▁', 'ทําได้', 'หรือไม่', '?', '▁', 'ตอบ', 'แค่', 'ได้', '▁หรือ', '▁', 'ไม่ได้', '▁ถ้า', 'ได้', 'ให้', 'ตอบ', '▁1', '▁และ', 'ถ้า', 'ไม่ได้', 'ให้', 'ตอบ', '▁0', '▁[', 'S', 'EP', ']']


In [ ]:
#give index number for tokenized word

from tqdm import tqdm
from tqdm import  tqdm_notebook
#uncomment all three lines below to convert to ids
input_ids=[]
for i in tqdm_notebook(range(len(tokenized_texts))):
  input_ids.append(tokenizer.convert_tokens_to_ids(tokenized_texts[i]))

/tmp/ipykernel_94/1317061965.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(tokenized_texts))):


  0%|          | 0/4429 [00:00<?, ?it/s]

In [ ]:
labels

In [ ]:
loss

In [ ]:
#standardize code

from keras.preprocessing.sequence import pad_sequences

MAX_LEN = 256
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [ ]:
#Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

#Machine learning models shouldn't waste effort processing padding tokens.  The attention mask lets the model know which tokens to focus on and which to ignore.

In [ ]:
from sklearn.model_selection import train_test_split
# Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels,random_state=56, test_size=0.2)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=56, test_size=0.2)

In [ ]:
#Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [ ]:
#data to loader
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 16

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop,
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
## Define model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
!pip install pytorch_transformers
from pytorch_transformers import *

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 2.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 10.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 57.8 MB/s eta 0:00:0000:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.69
    Uninstalling botocore-1.34.69:
      Successfully uninstalled botocore-1.34.69
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.29.165 which is incompatible.


In [ ]:
pip install torch_optimizer


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 943.7 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
from torch_optimizer import RAdam # Install via: pip install torch_optimizer


In [ ]:
#meta parameter

lr = 2e-5
max_grad_norm = 1.0
num_total_steps = 1000
num_warmup_steps = 100
warmup_proportion = float(num_warmup_steps) / float(num_total_steps)  # 0.1


### In PyTorch-Transformers, optimizer and schedules are splitted and instantiated like this:
optimizer = RAdam(model.parameters(), lr=lr)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_total_steps)

In [ ]:
len(train_dataloader)

222

In [ ]:
model.to("cuda")

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [ ]:
total_step = len(train_dataloader)

# Store our loss and accuracy for plotting
train_loss_set = []


epochs = 2

# trange is a tqdm wrapper around the normal python range
for epoch in tqdm_notebook(range(epochs)):



    # Training
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    # Train the data for one epoch
    for i, batch in enumerate(train_dataloader):
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels = batch
      # Forward pass
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
      loss = outputs[0]
      train_loss_set.append(loss.item())
      # Backward pass
      loss.backward()
      # Update parameters and take a step using the computed gradient
      optimizer.step()
      scheduler.step()
      optimizer.zero_grad()
      if (i) % 50 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, epochs, i+1, total_step, loss.item()))


/tmp/ipykernel_94/2779366120.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm_notebook(range(epochs)):


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch [1/2], Step [1/222], Loss: 0.7526
Epoch [1/2], Step [51/222], Loss: 0.6981
Epoch [1/2], Step [101/222], Loss: 0.5608
Epoch [1/2], Step [151/222], Loss: 0.6486
Epoch [1/2], Step [201/222], Loss: 0.5211
Epoch [2/2], Step [1/222], Loss: 0.6499
Epoch [2/2], Step [51/222], Loss: 0.6817
Epoch [2/2], Step [101/222], Loss: 0.4643
Epoch [2/2], Step [151/222], Loss: 0.4242
Epoch [2/2], Step [201/222], Loss: 0.2265


In [ ]:
torch.save(model, "kuymodel")

In [ ]:
total_step = len(train_dataloader)

# Store our loss and accuracy for plotting
train_loss_set = []


epochs = 2

# trange is a tqdm wrapper around the normal python range
for epoch in tqdm_notebook(range(epochs)):



    # Training
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    # Train the data for one epoch
    for i, batch in enumerate(train_dataloader):
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels = batch
      # Forward pass
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
      loss = outputs[0]
      train_loss_set.append(loss.item())
      # Backward pass
      loss.backward()
      # Update parameters and take a step using the computed gradient
      optimizer.step()
      scheduler.step()
      optimizer.zero_grad()
      if (i) % 50 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, epochs, i+1, total_step, loss.item()))


#ไม่ได้ใช้

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
svm_model = SVC(kernel='linear')

In [ ]:
len(X_train)

In [ ]:
# Feature Extraction (for SVM)
X_train_svm = tfidf_vectorizer.fit_transform(X_train)
X_test_svm = tfidf_vectorizer.transform(X_test)

# Train SVM
svm_model.fit(X_train_svm, y_train)

# Evaluate SVM
svm_predictions = svm_model.predict(X_test_svm)
svm_f1_macro = f1_score(y_test, svm_predictions, average='macro')  # F1-score (macro average)
print(f"SVM F1-Macro: {svm_f1_macro}")

In [ ]:
print(models)

In [ ]:
from transformers import AutoTokenizer, TFXLMRobertaForMaskedLM
import tensorflow as tf

# Load the XLM-RoBERTa model
model = models[1]
tokenizer = tokenizers[1]  # Assuming you want to use the tokenizer corresponding to the XLM-RoBERTa model

# Tokenize the input text
text = "The capital of France is <mask>."
inputs = tokenizer(text, return_tensors="tf")
print(type(inputs))
#print(tokenizer)
#print(inputs[0])

# Get the logits from the XLM-RoBERTa model
logits = model(inputs)[0]  # Output of the XLM-RoBERTa model

# Retrieve the index of the masked token
mask_token_index = tf.where(tf.equal(inputs["input_ids"], tokenizer.mask_token_id))

# Extract the logits corresponding to the masked token
selected_logits = tf.gather_nd(logits, mask_token_index)

# Get the predicted token ID
predicted_token_id = tf.math.argmax(selected_logits, axis=-1)

# Decode the predicted token ID
predicted_token = tokenizer.decode(predicted_token_id)
print(predicted_token)


In [ ]:
type(X_test)

In [ ]:
# Ensemble Prediction Function
def predict_ensemble(models, tokenizers, svm_model, X, for_svm=False):
  predictions = []

  for text in X:
    transformer_preds = []
    for model, tokenizer in zip(models, tokenizers):
      inputs = tokenizer(text, return_tensors="tf")
      print(inputs)

      # Assuming your models have a .predict() method for raw predictions
      model_preds = model(inputs).argmax(axis=-1)[0]
      transformer_preds.append(model_preds)

    svm_input = preprocess_text(text, None, for_svm=True)
    svm_pred = svm_model.predict(svm_input)[0]  # Get class prediction

    # Implement your ensemble logic (e.g., majority voting)
    if sum(transformer_preds) > len(transformer_preds) / 2 and svm_pred == 1:
      ensemble_prediction = 1
    else:
      ensemble_prediction = 0

    predictions.append(ensemble_prediction)

  return predictions


In [ ]:
# Prediction on Test Set
ensemble_predictions = predict_ensemble(models, tokenizers, svm_model, X_test.tolist(), for_svm=True)

# Evaluation using F1-score
f1_macro = f1_score(y_test, ensemble_predictions, average='macro')
print(f"Ensemble F1-Macro: {f1_macro}")
